# Web page 자동화 매뉴얼 (on Mac) 공부

# 창원시 축구장 예약 자동화 (자동 기입방지 때문에 반자동)

## Chrome driver 설치
```zsh
brew install chromedriver
```

## Selenium 설치
```zsh
pip3 install selenium
```

In [1]:
import sys
print(sys.version)

3.7.10 (default, Feb 26 2021, 10:16:00) 
[Clang 10.0.0 ]


In [2]:
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pause

## 운동장 자동 예약하기
- ... 로그인은 어떻게 하지..?

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Chrome
driver = webdriver.Chrome()

# Firefox
# driver = webdriver.Firefox()
# Intenet explorer
# driver = webdriver.Ie()

### 경기 날짜 규칙 파악 및 달력 넘기기 알고리즘 생성
- 마감 전 규칙
```python
# 3월 24일 4주차 수요일 3번째 #rev_calendar > li:nth-child(5) > div.days24 > div > a:nth-child(3)
# 3월 26일 4주차 금요일 2번째 #rev_calendar > li:nth-child(5) > div.days26 > div > a
# 3월 30일 5주차 화요일 2번째 #rev_calendar > li:nth-child(6) > div.days30 > div > a
# 4월 02일 1주차 금요일 3번째 #rev_calendar > li:nth-child(2) > div.days02 > div > a:nth-child(3)
```
- 마감 후 규칙
```python
# 3월 27일 4주차 토요일 4번째 #rev_calendar > li:nth-child(5) > div.days27.Sat > div > span:nth-child(4)
# 3월 26일 4주차 금요일 3번째 #rev_calendar > li:nth-child(5) > div.days26 > div > span:nth-child(4)
# 3월 28일 5주차 일요일 5번째 #rev_calendar > li:nth-child(6) > div.days28.Sun > div > span:nth-child(5)
# 4월 03일 1주차 토요일 1번째 #rev_calendar > li:nth-child(2) > div.days03.Sat > div > span:nth-child(1)
```
- **대표적인 공식**
```python
#rev_calendar > li:nth-child(?) > div.days??.Sat > div > a
```

- **정리**  
우리가 목표로 하는 시간은 매주 토요일 2번째 시간!!!!
    1. **오늘은 몇 주차 인가**  > 달력 상 n 주차라면  #rev_calendar > li:nth-child(n+1)
    2. **토, 일요일은 기록된다** > ```#rev_calendar > li:nth-child(n+1) > div.days00.Sat >```
    3. **2번째 타임을 노린다**  > ```#rev_calendar > li:nth-child(n+1) > div.days00.Sat > div > a```
    - **달력 다음 페이지? 지금 페이지?**
        - 
    

In [4]:
from math import floor
from datetime import datetime, timedelta

# 지금 몇 주차인가? n 값

def new_weekday(x) : 
    # Sun : 0 to Sat :6
    newweekday = x.weekday() + 1
    if newweekday == 7 :
        newweekday = 0
        
    return newweekday

def n_th_week(x) : 
    now = x
    first_day_of_month = now - timedelta(now.day-1)
    first_day_of_month.weekday() #0 -> 월요일 
    re_weekday = new_weekday(first_day_of_month)

    diff = re_weekday - 1
    return floor((now.day + diff)/7)+1



In [5]:
# 0. 토요일이 되는 순간 for loop 탈출하도록 만들기
#while 1:
#    #if saturday, BREAK, and start Below CODE
#    if datetime.now().weekday() == 5 : break

# 1. 오늘로 부터 14일 뒤를 예약해야한다.
day_plus_14 = datetime.now() + timedelta(days = 14)

# 2. 2주 뒤는 그 달의 몇 주차? => n
n = n_th_week(day_plus_14)
_nth_child = 'li:nth-child({})'.format(n+1)

# 3. 오늘이 토요일이긴 한데, 몇 일? (무조건 토요일만 뽑을것이기 때문에 상관 무)
_days00_Sat = 'div.days' + str(day_plus_14.day).zfill(2) + '.Sat'

# 4. 두번째 타임 예약 문자열
_2nd_time = 'a:nth-child(2)'

# 모든 문자열
_When_CSS = '#rev_calendar > {} > {} > {}'.format(_nth_child, _days00_Sat, _2nd_time)

print(_When_CSS)

#rev_calendar > li:nth-child(3) > div.days09.Sat > a:nth-child(2)


- **달력 넘겨야해? 말아야해?**
    - 달력에 표시되는 날은 현재날 +3 ~ +14일 
    - 예약을 해야하는 날은 +14일

# 예약 매뉴얼
## 시작 - 필요한 모듈 임포트 및 함수 생성

In [6]:
import time
import pause
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from math import floor
from datetime import datetime, timedelta

# 지금 몇 주차인가? n 값
def new_weekday(x) : 
    # Sun : 0 to Sat :6
    newweekday = x.weekday() + 1
    if newweekday == 7 :
        newweekday = 0
        
    return newweekday

def n_th_week(x) : 
    now = x
    first_day_of_month = now - timedelta(now.day-1)
    first_day_of_month.weekday() #0 -> 월요일 
    re_weekday = new_weekday(first_day_of_month)

    diff = re_weekday - 1
    return floor((now.day + diff)/7)+1


# 새 컨트롤 가능한 새창 업로드
# Chrome
driver = webdriver.Chrome()

# Firefox
# driver = webdriver.Firefox()
# Intenet explorer
# driver = webdriver.Ie()



In [7]:
url = 'https://www.changwon.go.kr/reservation/main.do'

wait = WebDriverWait(driver, timeout=10)
driver.get(url)

## 로그인 후 다음 클릭
**12시 되기 5초전에 다음 구동**

In [13]:
url = 'https://www.changwon.go.kr/reservation/request/list.do?searchFcltyCd=oas&mId=0344010201'

wait = WebDriverWait(driver, timeout=10)
driver.get(url)

# 0. 토요일이 되는 순간 for loop 탈출하도록 만들기
while 1:
    #if saturday, BREAK, and start Below CODE
    if datetime.now().weekday() == 5 : break

# 1. 오늘로 부터 14일 뒤를 예약해야한다.
day_plus_14 = datetime.now() + timedelta(days = 14)

# 2. 2주 뒤는 그 달의 몇 주차? => n
n = n_th_week(day_plus_14)
_nth_child = 'li:nth-child({})'.format(n+1)

# 3. 오늘이 토요일이긴 한데, 몇 일? (무조건 토요일만 뽑을것이기 때문에 상관 무)
_days00_Sat = 'div.days' + str(day_plus_14.day).zfill(2) + '.Sat'

# 4. 두번째 타임 예약 문자열
_2nd_time = 'a:nth-child(2)'

# 모든 문자열
_When_CSS = '#rev_calendar > {} > {} > {}'.format(_nth_child, _days00_Sat, _2nd_time)

print(_When_CSS)

# 3월 28일 같은 경우에는 3월에 더이상 표시할 예약칸이 없다. 이 경우에는 바로 4월 칸으로 넘어가나? 이걸 보고 움직여야 한다.
# 일단은, 14일 뒤가 다음달인 경우 페이지를 넘기는 작업을 해야한다
now = datetime.now()
if day_plus_14.month > now.month : 
    next_month = driver.find_element_by_css_selector('#calendar_next')
    next_month.click()
    

##reserv = driver.find_element_by_css_selector('#rev_calendar > li:nth-child(2) > div.days02 > div > a:nth-child(2)')
##reserv.click()

reserv = driver.find_element_by_css_selector(_When_CSS)
reserv.click()

agree = driver.find_element_by_css_selector('#write > div.theme-radio.pb20 > label')
agree.click()

agree2 = driver.find_element_by_css_selector('#write > div.txt-center.pt30.pb50 > button.btn.point')
agree2.click()

Team_name = driver.find_element_by_css_selector('#groupNm')
Team_name.send_keys('MUTO')

phone_num = driver.find_element_by_css_selector('#phone')
phone_num.send_keys('010-3678-3547')

obj = driver.find_element_by_css_selector('#etc')
obj.send_keys('축구 경기 및 훈련')

cnt1 = driver.find_element_by_css_selector('#cnt1')
cnt1.send_keys('12')

cnt2 = driver.find_element_by_css_selector('#cnt2')
cnt2.send_keys('0')

cnt3 = driver.find_element_by_css_selector('#cnt3')
cnt3.send_keys('12')

agree3 = driver.find_element_by_css_selector('#agree')
agree3.click()

agree4 = driver.find_element_by_css_selector('#agree2')
agree4.click()

# 우편번호
zipcode = driver.find_element_by_css_selector('#writeForm > table > tbody > tr:nth-child(8) > td > fieldset > button')
zipcode.click()

driver.switch_to_window(driver.window_handles[1])  
#driver.get_window_position(driver.window_handles[1])

search = driver.find_element_by_css_selector('#keyword')
search.send_keys('창이대로707번길 7-9')

search_ent = driver.find_element_by_css_selector('#addrSearch')
search_ent.click()

pause.seconds(0.7)
select_adr = driver.find_element_by_css_selector('#addrInfo > table > tbody > tr > td:nth-child(2) > a')
select_adr.click()

detailAddr = driver.find_element_by_css_selector('#detailAddr')
detailAddr.send_keys('203호')

insertAddr = driver.find_element_by_css_selector('#insertForm > div > a')
insertAddr.click()

driver.switch_to_window(driver.window_handles[0])  
#driver.get_window_position(driver.window_handles[0])

### 자동기입방지 문자 입력하기
STRS = input('자동기입방지문구 : ')
captchaStr = driver.find_element_by_css_selector('#captchaStr')
captchaStr.send_keys(STRS)

### 마지막 신청하기
final = driver.find_element_by_css_selector('#conts > div.txt-center.pt30.pb50 > a.btn.point')
final.click()

#rev_calendar > li:nth-child(3) > div.days10.Sat > a:nth-child(2)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#rev_calendar > li:nth-child(3) > div.days10.Sat > a:nth-child(2)"}
  (Session info: chrome=89.0.4389.90)


### 예약 날짜 실패시 바로 ㄱ ㄱ

In [11]:
url = 'https://www.changwon.go.kr/reservation/request/list.do?searchFcltyCd=oas&mId=0344010201'

wait = WebDriverWait(driver, timeout=10)
driver.get(url)

# 0. 토요일이 되는 순간 for loop 탈출하도록 만들기
#while 1:
#    #if saturday, BREAK, and start Below CODE
#    if datetime.now().weekday() == 5 : break

# 1. 오늘로 부터 14일 뒤를 예약해야한다.
day_plus_14 = datetime.now() + timedelta(days = 14)

# 2. 2주 뒤는 그 달의 몇 주차? => n
n = n_th_week(day_plus_14)
_nth_child = 'li:nth-child({})'.format(n+1)

# 3. 오늘이 토요일이긴 한데, 몇 일? (무조건 토요일만 뽑을것이기 때문에 상관 무)
_days00_Sat = 'div.days' + str(day_plus_14.day).zfill(2) + '.Sat'

# 4. 두번째 타임 예약 문자열
_2nd_time = 'a:nth-child(2)'

# 모든 문자열
_When_CSS = '#rev_calendar > {} > {} > {}'.format(_nth_child, _days00_Sat, _2nd_time)

print(_When_CSS)

# 3월 28일 같은 경우에는 3월에 더이상 표시할 예약칸이 없다. 이 경우에는 바로 4월 칸으로 넘어가나? 이걸 보고 움직여야 한다.
# 일단은, 14일 뒤가 다음달인 경우 페이지를 넘기는 작업을 해야한다
now = datetime.now()
if day_plus_14.month > now.month : 
    next_month = driver.find_element_by_css_selector('#calendar_next')
    next_month.click()
        

#rev_calendar > li:nth-child(3) > div.days09.Sat > a:nth-child(2)


In [14]:
agree = driver.find_element_by_css_selector('#write > div.theme-radio.pb20 > label')
agree.click()

agree2 = driver.find_element_by_css_selector('#write > div.txt-center.pt30.pb50 > button.btn.point')
agree2.click()

Team_name = driver.find_element_by_css_selector('#groupNm')
Team_name.send_keys('MUTO')

phone_num = driver.find_element_by_css_selector('#phone')
phone_num.send_keys('010-3678-3547')

obj = driver.find_element_by_css_selector('#etc')
obj.send_keys('축구 경기 및 훈련')

cnt1 = driver.find_element_by_css_selector('#cnt1')
cnt1.send_keys('12')

cnt2 = driver.find_element_by_css_selector('#cnt2')
cnt2.send_keys('0')

cnt3 = driver.find_element_by_css_selector('#cnt3')
cnt3.send_keys('12')

agree3 = driver.find_element_by_css_selector('#agree')
agree3.click()

agree4 = driver.find_element_by_css_selector('#agree2')
agree4.click()

# 우편번호
zipcode = driver.find_element_by_css_selector('#writeForm > table > tbody > tr:nth-child(8) > td > fieldset > button')
zipcode.click()

driver.switch_to_window(driver.window_handles[1])  
#driver.get_window_position(driver.window_handles[1])

search = driver.find_element_by_css_selector('#keyword')
search.send_keys('창이대로707번길 7-9')

search_ent = driver.find_element_by_css_selector('#addrSearch')
search_ent.click()

pause.seconds(0.7)
select_adr = driver.find_element_by_css_selector('#addrInfo > table > tbody > tr > td:nth-child(2) > a')
select_adr.click()

detailAddr = driver.find_element_by_css_selector('#detailAddr')
detailAddr.send_keys('203호')

insertAddr = driver.find_element_by_css_selector('#insertForm > div > a')
insertAddr.click()

driver.switch_to_window(driver.window_handles[0])  
#driver.get_window_position(driver.window_handles[0])

### 자동기입방지 문자 입력하기
STRS = input('자동기입방지문구 : ')
captchaStr = driver.find_element_by_css_selector('#captchaStr')
captchaStr.send_keys(STRS)

### 마지막 신청하기
final = driver.find_element_by_css_selector('#conts > div.txt-center.pt30.pb50 > a.btn.point')
final.click()

/Users/gangtaro/opt/anaconda3/envs/se3.7/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: use driver.switch_to.window instead
/Users/gangtaro/opt/anaconda3/envs/se3.7/lib/python3.7/site-packages/ipykernel_launcher.py:54: DeprecationWarning: use driver.switch_to.window instead


자동기입방지문구 : 4c5r3
